# Linear Regression Example

Let's walk through the steps of the official documentation example. Doing this will help your ability to read from the documentation, understand it, and then apply it to your own problems (the upcoming Consulting Project).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
# Load training data
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

Interesting! We haven't seen libsvm formats before. In fact the aren't very popular when working with datasets in Python, but the Spark Documentation makes use of them a lot because of their formatting. Let's see what the training data looks like:

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

This is the format that Spark expects. Two columns with the names "label" and "features". 

The "label" column then needs to have the numerical label, either a regression numerical value, or a numerical value that matches to a classification grouping. Later on we will talk about unsupervised learning algorithms that by their nature do not use or require a label.

The feature column has inside of it a vector of all the features that belong to that row. Usually what we end up doing is combining the various feature columns we have into a single 'features' column using the data transformations we've learned about.

Let's continue working through this simple example!

In [6]:
# These are the default values for the featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol='features', labelCol='label', 
                      predictionCol='prediction')

# You could also pass in additional parameters for regularization, do the reading 
# in ISLR to fully understand that, after that its just some simple parameter calls.
# Check the documentation with Shift+Tab for more info!

In [7]:
# Fit the model
lrModel = lr.fit(training)

In [24]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coefficients: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]


Intercept:0.14228558260358093


There is a summary attribute that contains even more info!

In [25]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary

Lots of info, here are a few examples:

In [28]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176538|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
|  2.122807193191233|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154


## Train/Test Splits

But wait! We've commited a big mistake, we never separated our data set into a training and test set. Instead we trained on ALL of the data, something we generally want to avoid doing. Read ISLR and check out the theory lecture for more info on this, but remember we won't get a fair evaluation of our model by judging how well it does again on the same data it was trained on!

Luckily Spark DataFrames have an almost too convienent method of splitting the data! Let's see it:

In [29]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [30]:
# Pass in the split between training/test as a list.
# No correct, but generally 70/30 or 60/40 splits are used. 
# Depending on how much data you have and how unbalanced it is.
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [40]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
+----------

In [42]:
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
| -16.08565904102149|(10,[0,1,2,3,4,5,...|
| -15.86200932757056|(10,[0,1,2,3,4,5,...|
| -15.72351561304857|(10,[0,1,2,3,4,5,...|
|-15.375857723312297|(10,[0,1,2,3,4,5,...|
|-15.056482974542433|(10,[0,1,2,3,4,5,...|
|-13.867087895158768|(10,[0,1,2,3,4,5,...|
|-13.420594775890757|(10,[0,1,2,3,4,5,...|
| -13.15333560636553|(10,[0,1,2,3,4,5,...|
|-12.491442077546413|(10,[0,1,2,3,4,5,...|
|-12.130353212287929|(10,[0,1,2,3,4,5,...|
|-12.094351278535258|(10,[0,1,2,3,4,5,...|
|-11.904986902675114|(10,[0,1,2,3,4,5,...|
|-11.857350365429426|(10,[0,1,2,3,4,5,...|
|-11.615775265015627|(10,[0,1,2,3,4,5,...|
+----------

In [49]:
unlabeled_data = test_data.select('features')

In [50]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



Now we only train on the train_data

In [51]:
correct_model = lr.fit(train_data)

Now we can directly get a .summary object using the evaluate method:

In [56]:
test_results = correct_model.evaluate(test_data)

In [68]:
test_results.predictions.show()

+-------------------+--------------------+--------------------+
|              label|            features|          prediction|
+-------------------+--------------------+--------------------+
|-26.736207182601724|(10,[0,1,2,3,4,5,...| -3.0008828649041503|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|   2.624844444892833|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|  -3.157976474278272|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|  2.0099996028372655|
| -19.66731861537172|(10,[0,1,2,3,4,5,...| -0.0514086205450508|
|-17.065399625876015|(10,[0,1,2,3,4,5,...| 0.40984367412354844|
| -16.08565904102149|(10,[0,1,2,3,4,5,...| -0.8662362874387104|
| -15.86200932757056|(10,[0,1,2,3,4,5,...|  2.8514839794859297|
| -15.72351561304857|(10,[0,1,2,3,4,5,...|   3.238496449878981|
|-15.375857723312297|(10,[0,1,2,3,4,5,...|  1.3123968536973585|
|-15.056482974542433|(10,[0,1,2,3,4,5,...|  2.4722093477541236|
|-13.867087895158768|(10,[0,1,2,3,4,5,...| -1.7236964326091746|
|-13.420594775890757|(10,[0,1,2,3,4,5,..

In [53]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

+-------------------+
|          residuals|
+-------------------+
|-23.735324317697575|
|-25.574670381088907|
| -19.67948394264107|
| -21.88299064090567|
| -19.61590999482667|
|-17.475243299999562|
| -15.21942275358278|
| -18.71349330705649|
| -18.96201206292755|
|-16.688254577009655|
|-17.528692322296557|
|-12.143391462549594|
|-13.697505078174897|
|-12.934258314196214|
|-15.077975986069932|
|-10.998905393217235|
| -11.77690655032861|
|   -11.245327464793|
|-11.851357107126017|
|-14.211835091367506|
+-------------------+
only showing top 20 rows

RMSE: 10.56841456442691


Well that is nice, but realistically we will eventually want to test this model against unlabeled data, after all, that is the whole point of building the model in the first place. We can again do this with a convenient method call, in this case, transform(). Which was actually being called within the evaluate() method. Let's see it in action:

In [54]:
predictions = correct_model.transform(unlabeled_data)

In [55]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -3.0008828649041503|
|(10,[0,1,2,3,4,5,...|   2.624844444892833|
|(10,[0,1,2,3,4,5,...|  -3.157976474278272|
|(10,[0,1,2,3,4,5,...|  2.0099996028372655|
|(10,[0,1,2,3,4,5,...| -0.0514086205450508|
|(10,[0,1,2,3,4,5,...| 0.40984367412354844|
|(10,[0,1,2,3,4,5,...| -0.8662362874387104|
|(10,[0,1,2,3,4,5,...|  2.8514839794859297|
|(10,[0,1,2,3,4,5,...|   3.238496449878981|
|(10,[0,1,2,3,4,5,...|  1.3123968536973585|
|(10,[0,1,2,3,4,5,...|  2.4722093477541236|
|(10,[0,1,2,3,4,5,...| -1.7236964326091746|
|(10,[0,1,2,3,4,5,...| 0.27691030228414015|
|(10,[0,1,2,3,4,5,...|-0.21907729216931587|
|(10,[0,1,2,3,4,5,...|   2.586533908523519|
|(10,[0,1,2,3,4,5,...| -1.1314478190706934|
|(10,[0,1,2,3,4,5,...| -0.3174447282066493|
|(10,[0,1,2,3,4,5,...| -0.6596594378821141|
|(10,[0,1,2,3,4,5,...|-0.00599325830340...|
|(10,[0,1,2,3,4,5,...|  2.596059

Okay, so this data is a bit meaningless, so let's explore this same process with some data that actually makes a little more intuitive sense!